# Granularity investigation

In this notebook, we run our solver on old data with different granularity.

Please check `transplants_investigation.ipynb` before running this notebook.

## Imports

In [ ]:
import pandas as pd
import os
from typing import List
import re
import numpy as np
import sys
import math
import time
import logging

In [ ]:
sys.path.insert(0, "../..")

from local_testing_utilities.notebook_utils.pairing_data import parse_pairing_data
from local_testing_utilities.notebook_utils.survival_data import parse_survival_data
from txmatching.utils.blood_groups import BloodGroup
from txmatching.utils.country_enum import Country
from txmatching.patients.patient import Donor, Recipient
from txmatching.patients.patient_parameters import PatientParameters
from tests.test_utilities.hla_preparation_utils import (create_antibodies, create_antibody,
                                                        create_hla_typing)
from tests.test_utilities.prepare_app_for_tests import DbTests
from txmatching.patients.patient import TxmEvent
from txmatching.utils.enums import TxmEventState
from txmatching.configuration.configuration import Configuration
from txmatching.solve_service.solve_from_configuration import solve_from_configuration

## Load data

In [ ]:
df_all_patients = parse_pairing_data('data/KDP-processed', 'data/patients_list_recipientID.csv', remove_single_donors=False)

In [ ]:
df_survival = parse_survival_data('data/LD_kidney_survival.csv')
df_survival_summary = pd.read_pickle('data/survival_summary.pkl')

In [ ]:
df_patients_with_recipient_id = pd.read_csv('data/patients_list_recipientID.csv')
df_transplanted_donors = pd.read_excel('data/transplanted_donors.xlsx', index_col=None)

## 💁‍♂️ Run solver on data with various granularity

To all patients records, we join transplants if found for both donors and recipients

In [ ]:
df_all_patients_with_survival = df_all_patients\
    .join(df_survival_summary.set_index('RecipientID').add_suffix('_recip_surv'), on='recipient_id')\
    .join(df_transplanted_donors.set_index('donor_name')['target_recipient_id'], on='donor_name')\
    .join(df_survival_summary.set_index('RecipientID').add_suffix('_donor_surv'), on='target_recipient_id')\
    .assign(recip_has_transplant=lambda df: df.delay_recip_surv.notnull())\
    .assign(donor_has_transplant=lambda df: df.delay_donor_surv.notnull())
# df_all_patients_with_survival.head()
# df_all_patients_with_survival[lambda df: df.donor_has_transplant]

Now we define function that returns patients that would be in txm event with different granularity.

For given granularity, each event has patients from the originla event plus patients from $granularity - 1$ previous events that have been transplanted.

In [ ]:
def get_recipients_for_granularity(txm_event, granularity):
    df_patients_for_granularity = df_all_patients_with_survival.loc[
        lambda df:
        (df.txm_event == txm_event) | 
        ((df.txm_event > txm_event - granularity) & (df.txm_event < txm_event) & df.recip_has_transplant)
    ]
    df_patients_for_granularity = df_patients_for_granularity.drop_duplicates(subset=['recipient_id'], keep='last')#.reset_index()#.set_index('recipient_id')
    df_patients_for_granularity = df_patients_for_granularity.loc[lambda df: df.recipient_id.notnull()]
    return df_patients_for_granularity

#get_recipients_for_granularity(txm_event=26, granularity=2)

In [ ]:
def get_donors_for_granularity(txm_event, granularity):
    df_patients_for_granularity = df_all_patients_with_survival.loc[
        lambda df:
        (df.txm_event == txm_event) | 
        ((df.txm_event > txm_event - granularity) & (df.txm_event < txm_event) & df.donor_has_transplant)
    ]
    df_patients_for_granularity = df_patients_for_granularity.drop_duplicates(subset=['donor_name'], keep='last')#.reset_index()#.set_index('donor_name')
    return df_patients_for_granularity

#get_donors_for_granularity(txm_event=26, granularity=2)

Before we run the solver, we make some config

In [ ]:
# Initialize db
test = DbTests()
test.setUp()

In [ ]:
# test.tearDown()

In [ ]:
logger = logging.getLogger()
logger.setLevel('WARN')

Now we can run the solver. We define several functions for that.

In [ ]:
def row_to_recipient(db_id, row):
    recipient_typization = row.recipient_typization
    recipient_antibodies = row.recipient_luminex_2
    
    recipient_blood_group = row.recipient_blood_group
    recipient_acceptable_blood = row.recipient_acceptable_blood
    
    db_id = row.recipient_id
    
    if recipient_typization == '':
        print(f"Problem with recipient empty typization {db_id}")
        return None
    
    recipient_typing = recipient_typization.split(" ")
    recipient_antibodies = recipient_antibodies.split(" ") if recipient_antibodies != '' else []
    recipient_acceptable_blood = recipient_acceptable_blood.split(" ") if recipient_acceptable_blood != '' else []
    
    recipient = Recipient(
        db_id=db_id,
        acceptable_blood_groups=[BloodGroup(group) for group in recipient_acceptable_blood],
        related_donor_db_id=db_id,
        medical_id=f'recipient_{db_id}',
        parameters=PatientParameters(
            blood_group=BloodGroup(recipient_blood_group),
            country_code=Country.CZE,
            hla_typing=create_hla_typing(recipient_typing)
        ),
        hla_antibodies=create_antibodies([create_antibody(raw_code, 2000, 2000) for raw_code in recipient_antibodies])
    )
    
    return recipient

In [ ]:
def row_to_donor(db_id, row):
    donor_typization = row.donor_typization
    donor_blood_group = row.donor_blood_group
    donor_is_single = math.isnan(row.recipient_id)
    
    if not donor_is_single:
        db_id = row.recipient_id
    
    if donor_typization == '':
        print(f"Problem with donor empty typization {db_id}")
        return None
    
    donor_typing = donor_typization.split(" ")
    
    donor = Donor(
        db_id=db_id,
        medical_id=f'donor_{db_id}',
        related_recipient_db_id=db_id if not donor_is_single else None,
        parameters=PatientParameters(
            blood_group=BloodGroup(donor_blood_group),
            country_code=Country.CZE,
            hla_typing=create_hla_typing(
                donor_typing
            )
        )
    )
    
    return donor

In [ ]:
def compute_for_patients(df_donors, df_recipients, forbidden_donor_ids = set(), forbidden_recipient_ids = set()):
    donors = []
    recipients = []
    
    for index, row in df_donors.iterrows():
        maybe_donor = row_to_donor(int(index), row)
        if maybe_donor is None or maybe_donor.medical_id in forbidden_donor_ids:
            continue
        
        donors.append(maybe_donor)
    
    for index, row in df_recipients.iterrows():
        maybe_recipient = row_to_recipient(int(index), row)
        if maybe_recipient is None or maybe_recipient.medical_id in forbidden_recipient_ids:
            continue
        
        recipients.append(maybe_recipient)

    txm_event = TxmEvent(1, 'sample_event', None, TxmEventState.OPEN, donors, recipients)
    configuration = Configuration(
        max_number_of_matchings=1,
        max_cycle_length=50,
        max_sequence_length=50,
        # solver_constructor_name='AllSolutionsSolver'
    )
    pairing_result = solve_from_configuration(configuration, txm_event=txm_event)

    matchings_count = len(pairing_result.calculated_matchings_list)
    
    transplanted_donors_ids = set()
    transplanted_recipients_ids = set()
    
    if matchings_count > 0:
        matching = pairing_result.calculated_matchings_list[0]
        matching_pairs_count = len(matching.get_donor_recipient_pairs())
        #print(pairing_result.score_matrix)
        for pair in matching.get_donor_recipient_pairs():
            #print(f"{pair.donor.medical_id} -> {pair.recipient.medical_id}")
            transplanted_donors_ids.add(pair.donor.medical_id)
            transplanted_recipients_ids.add(pair.recipient.medical_id)
            
        
    else:
        matching_pairs_count = 0
    
    return matching_pairs_count, len(donors), len(recipients), transplanted_donors_ids, transplanted_recipients_ids

# df_donors = get_donors_for_granularity(txm_event=24, granularity=4)
# df_recipients = get_recipients_for_granularity(txm_event=24, granularity=4)
# display(df_donors)
# df_recipients[df_recipients.columns[:20]]

In [ ]:
d = []

for granularity in range(1, 5):
    for shift in range(granularity):
        forbidden_donor_ids = set()
        forbidden_recipient_ids = set()
        for txm_event in range(10, 31):
            if txm_event % granularity != shift:
                continue
            
            df_donors = get_donors_for_granularity(txm_event=txm_event, granularity=granularity)
            df_recipients = get_recipients_for_granularity(txm_event=txm_event, granularity=granularity)

            donors_count = len(df_donors.index)
            recipients_count = len(df_recipients.index)

            print(f"Computing matching for txm_event {txm_event} and granularity {granularity} ({donors_count} donors, {recipients_count} recipients, forbidden={len(forbidden_donor_ids)}/{len(forbidden_recipient_ids)})", end=" ")
            start = time.time()
            matching_pairs_count, valid_donors, valid_recipients, transplanted_donors_ids, transplanted_recipients_ids = \
                compute_for_patients(df_donors, df_recipients, forbidden_donor_ids, forbidden_recipient_ids)
            elapsed_time = time.time() - start
            print(f"-> {matching_pairs_count} transplants found ({elapsed_time:.2f} seconds)")

            d.append({
                'txm_event': txm_event,
                'granularity': granularity,
                'donors_count': donors_count,
                'recipients_count': recipients_count,
                'valid_donors': valid_donors,
                'valid_recipients': valid_recipients,
                'forbidden_count': len(forbidden_donor_ids),
                'matching_pairs_count': matching_pairs_count,
                'matching_pairs_count_normalized': matching_pairs_count / granularity,
                'elapsed_time': elapsed_time
            })
            
            forbidden_donor_ids.update(transplanted_donors_ids)
            forbidden_recipient_ids.update(transplanted_recipients_ids)

df_granularity_results = pd.DataFrame(d).sort_values(by=['txm_event'])
df_granularity_results

In [ ]:
df_granularity_results.to_csv('data/granularity_results.csv')

### Results

Everything is computed now. Lets show some plots

In [ ]:
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values=['donors_count']).plot(ylabel='Donors count')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='recipients_count').plot(ylabel='Recipients count')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='valid_donors').plot(ylabel='Valid donors')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='forbidden_count').plot(ylabel='Forbidden count')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='matching_pairs_count').plot(ylabel='matching_pairs_count')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='matching_pairs_count_normalized').plot(ylabel='matching_pairs_count_normalized')
df_granularity_results.pivot_table(index='txm_event', columns='granularity', values='elapsed_time').plot(ylabel='elapsed_time (s)')

for granularity in range(1, 5):
    df_granularity_results\
        .loc[lambda df: df.granularity == granularity]\
        .set_index('txm_event')\
        [['donors_count', 'recipients_count', 'matching_pairs_count']]\
        .plot(title=f'granularity = {granularity}')
        #.assign(ratio=lambda df: df.matching_pairs_count / df.patients_count)\
        #.assign(diff=lambda df: - df.matching_pairs_count + df.patients_count)\

#### Compare with original transplant count
Now, lets suppose granularity = 1 and compare the computed results with original transplant count that we know using survival data.

Note: The original transplant count was moved to `transplants_investigation`.

Our algorthm found less transplants because it utilizes czech patients only.

#### Group by 1 year

In [ ]:
df_results_per_year = df_granularity_results\
    .assign(year=lambda df: (df.txm_event-10)//4)\
    .assign(event_in_year=lambda df: (df.txm_event-10)%4)
df_results_per_year['granularity_1'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 1 else 0, axis=1)
df_results_per_year['granularity_2_shift_0'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 2 and s.event_in_year in [0, 2] else 0, axis=1)
df_results_per_year['granularity_2_shift_1'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 2 and s.event_in_year in [1, 3] else 0, axis=1)
df_results_per_year = df_results_per_year.assign(granularity_2_mean=lambda df: (df.granularity_2_shift_0 + df.granularity_2_shift_1) / 2)
df_results_per_year['granularity_4_shift_0'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 4 and s.event_in_year == 0 else 0, axis=1)
df_results_per_year['granularity_4_shift_1'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 4 and s.event_in_year == 1 else 0, axis=1)
df_results_per_year['granularity_4_shift_2'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 4 and s.event_in_year == 2 else 0, axis=1)
df_results_per_year['granularity_4_shift_3'] = df_results_per_year.apply(lambda s: s.matching_pairs_count if s.granularity == 4 and s.event_in_year == 3 else 0, axis=1)
df_results_per_year = df_results_per_year.assign(granularity_4_mean=lambda df: (df.granularity_4_shift_0 + df.granularity_4_shift_1 + df.granularity_4_shift_2 + df.granularity_4_shift_3) / 4)

df_results_per_year = df_results_per_year.groupby(['year']).sum().reset_index()

# Do not show data from txm events that are not complete
df_results_per_year.loc[0, 'granularity_2_shift_0'] = None
df_results_per_year.loc[0, 'granularity_4_shift_0'] = None
df_results_per_year.loc[0, 'granularity_4_shift_1'] = None
df_results_per_year.loc[0, 'granularity_4_shift_2'] = None

df_results_per_year = df_results_per_year[[
    'granularity_1',
    'granularity_2_shift_0',
    'granularity_2_shift_1',
    'granularity_2_mean',
    'granularity_4_shift_0',
    'granularity_4_shift_1',
    'granularity_4_shift_2',
    'granularity_4_shift_3',
    'granularity_4_mean'
]]

df_results_per_year.plot(
    figsize=(15, 7), style=['b-','g-.','g--','g-','r-.','r--','r-o','r-.','r-'],
)

#### Overall number of transplants

In [ ]:
df_results_per_year.loc[1:].sum().plot.barh()
df_results_per_year.loc[1:].sum()[['granularity_1', 'granularity_2_mean', 'granularity_4_mean']]

### Show dependency between patient count and found transplants

In [ ]:
def get_random_patients(n):
    return df_all_patients.loc[lambda df: df.recipient_id.notnull()].drop_duplicates(subset=['recipient_id'], keep='last').sample(n=n)
# get_random_patients(n=5)

In [ ]:
d = []

for n in range(0, 40):
    print(f"Computing matching for ({n} patients)", end=" ")
    
    for i in range(20 if n < 10 else 5):
        df_random_patients = get_random_patients(n)
        
        patients_count = len(df_random_patients.index)
        
        start = time.time()
        matching_pairs_count, valid_donors, valid_recipients = compute_for_patients(df_random_patients, df_random_patients)
        elapsed_time = time.time() - start
        print(f", {matching_pairs_count} ({elapsed_time:.2f}s)", end=" ")
        
        d.append({
            'patients': patients_count,
            'valid_donors': valid_donors,
            'valid_recipients': valid_recipients,
            'matching_pairs_count': matching_pairs_count,
            'elapsed_time': elapsed_time
        })
    print()

df_ratio_results = pd.DataFrame(d)
df_ratio_results.to_csv('data/ratio_results.csv')
# df_ratio_results

In [ ]:
df_ratio_results.groupby('patients').agg({'matching_pairs_count': ['mean', 'std']}).plot(xlim=0, ylim=0)

In [ ]:
df_ratio_results.assign(ratio=lambda df: df.matching_pairs_count/df.patients).groupby('patients').agg({'ratio': ['mean', 'std']}).plot(xlim=0, ylim=0)

In [ ]:
df_ratio_results.groupby('patients').agg({'elapsed_time': ['mean', 'std']}).plot(xlim=0, ylim=0)